<a href="https://colab.research.google.com/github/bkee06/IBM_Capstone/blob/main/IBM_Data_Science_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IBM Data Science Capstone

This notebook contains content related to the final capstone project for IBM Applied Data Science. 

*Written by B Kee.* 


In [ ]:
import pandas as pd
import numpy as np


In [ ]:
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
CLIENT_ID = 'BUSKT154CN5GOLNB4MUXKSMASWB2J42A4YERNRN0TQPEJCGS' # your Foursquare ID
CLIENT_SECRET = 'ATWGVVUKGNTQOCYWSRHMSFK5BDP5UOZVUKTEYC11FQMNR0IR' # your Foursquare Secret
ACCESS_TOKEN = 'EPPIFUUZD0X1QAPUFPMENBBTMBMGEWMOQFRXQY4VBMZWL3FZ' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BUSKT154CN5GOLNB4MUXKSMASWB2J42A4YERNRN0TQPEJCGS
CLIENT_SECRET:ATWGVVUKGNTQOCYWSRHMSFK5BDP5UOZVUKTEYC11FQMNR0IR


In [4]:
address = '6 Rivett St, Hackett, ACT, Australia'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-35.2447202 149.166337


In [9]:
search_query = 'Italian'
radius = 5000
print(search_query + ' .... OK!')

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

Italian .... OK!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,5f823ad27a1eff27bf5f246e,The Italian Place Providore & Bottega,"[{'id': '4bf58dd8d48988d146941735', 'name': 'D...",v-1622707152,False,40 Mort St,-35.272568,149.131770,"[{'label': 'display', 'lat': -35.272568, 'lng'...",4413,2612,AU,Braddon,ACT,Australia,"[40 Mort St, Braddon ACT 2612]"
1,5c9ac58461f070002ce9e443,The Italian Place Enoteca,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1622707152,False,38 Mort St,-35.272866,149.131685,"[{'label': 'display', 'lat': -35.2728655169897...",4442,2612,AU,Canberra,ACT,Australia,"[38 Mort St, Canberra ACT 2612]"
2,4b8e3051f964a520d41a33e3,Italian and Sons,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1622707152,False,7 Lonsdale St,-35.275054,149.132423,"[{'label': 'display', 'lat': -35.2750535796285...",4572,2612,AU,Braddon,ACT,Australia,"[7 Lonsdale St, Braddon ACT 2612]"
3,6098a2ad2aebc24d5009ab8b,Italian Food P'sghetti Bar,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",v-1622707152,False,Shop 8/40 Marcus Clarke St,-35.283252,149.128660,"[{'label': 'display', 'lat': -35.2832522804833...",5488,2601,AU,Canberra,ACT,Australia,"[Shop 8/40 Marcus Clarke St, Canberra ACT 2601]"


In [10]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,The Italian Place Providore & Bottega,Deli / Bodega,40 Mort St,-35.272568,149.131770,"[{'label': 'display', 'lat': -35.272568, 'lng'...",4413,2612,AU,Braddon,ACT,Australia,"[40 Mort St, Braddon ACT 2612]",5f823ad27a1eff27bf5f246e
1,The Italian Place Enoteca,Italian Restaurant,38 Mort St,-35.272866,149.131685,"[{'label': 'display', 'lat': -35.2728655169897...",4442,2612,AU,Canberra,ACT,Australia,"[38 Mort St, Canberra ACT 2612]",5c9ac58461f070002ce9e443
2,Italian and Sons,Italian Restaurant,7 Lonsdale St,-35.275054,149.132423,"[{'label': 'display', 'lat': -35.2750535796285...",4572,2612,AU,Braddon,ACT,Australia,"[7 Lonsdale St, Braddon ACT 2612]",4b8e3051f964a520d41a33e3
3,Italian Food P'sghetti Bar,Fast Food Restaurant,Shop 8/40 Marcus Clarke St,-35.283252,149.128660,"[{'label': 'display', 'lat': -35.2832522804833...",5488,2601,AU,Canberra,ACT,Australia,"[Shop 8/40 Marcus Clarke St, Canberra ACT 2601]",6098a2ad2aebc24d5009ab8b


In [11]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map